In [1]:
from glovar import BRAND_LIST,IPHONE_LIST,STORAGE_LIST
import re

def get_phone_attribute_france(info_txt):
    """
    infomation txt: e.g "iPhone Apple APPLE IPHONE 12 128Go BLACK 5G"
    """
    def get_index1(lst=None, item=''):
        return [index for (index,value) in enumerate(lst) if value == item]

    brand, phone_type, storage_and_memory,color = None,None,None,None
    is_5g = False

    info = info_txt.strip().split()

    # get storage_and_memory info
    for storage in STORAGE_LIST:
        if storage in info_txt:
            storage_and_memory = storage
    
    # get brand and phone_type info 
    start,end = 0,0
    if "iphone" in info_txt:
        brand = "Apple"
        if "iphone se" in info_txt:
            phone_type = "iphone se"
        else:
            # start = info_txt.strip().split().index("iphone")
            start = get_index1(info_txt.strip().split(),"iphone")[-1]
            if storage_and_memory:
                end = info_txt.strip().split().index(storage_and_memory)
            else:
                end = start + 2
            # for t in IPHONE_LIST:
            #     if t in info_txt:
            #         phone_type = t
            phone_type = " ".join(info[start:end])
    elif "samsung" in info_txt:
        brand = "Samsung"
        start = info_txt.strip().split().index("samsung")
        if storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)
        else:
            end = start + 2
        phone_type = " ".join(info[start+1:end])
    elif "xiaomi" in info_txt:
        brand = "Xiaomi"
        # start = info_txt.strip().split().index("xiaomi")
        start = get_index1(info_txt.strip().split(),"xiaomi")[-1]
        if storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory) 
        else:
            end = start + 2
        phone_type = " ".join(info[start+1:end])
    elif "oppo" in info_txt:
        brand = "OPPO"
        start = info_txt.strip().split().index("oppo")
        if storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)
        else:
            end = start + 2
        phone_type = " ".join(info[start+1:end])

    # get is_5g info
    if "5g" in info_txt:
        is_5g = True

    # get color info
    # if storage_and_memory:
    #     color_start = info_txt.strip().split().index(storage_and_memory)
    #     color = " ".join(info_txt.strip().split()[color_start+1:color_start+2])
        
        # print("color:",color)

    return brand, phone_type, storage_and_memory, is_5g, color

def extract_and_save_france(book,sheet,file_name):
    sheet["C1"], sheet["D1"], sheet["E1"], sheet["F1"],sheet["G1"] = "brand", "phone_type", "storage_and_memory", "is_5g", "color"
    sheet["H1"] = "price"
    for i,row in enumerate(sheet.values):
        source_txt, price = row[0],row[1]

        if i > 0 :
            txt = source_txt.lower()
            txt = txt.strip().replace("gold","")
            txt = txt.strip().replace("go","gb")
            txt = txt.strip().replace("to","tb")
            txt = txt.strip().replace(" gb","gb")
            txt = txt.strip().replace(" tb","tb")
            txt = txt.strip().replace("\n"," ")

            price =price.strip().replace(" €*","")
            price =price.strip().replace(" ","")
            try:
                price = float(price.replace(",",".")) #这么写有没有问题！！！！！？？/、、？T_T
            except:
                continue
            # print(txt, price)

            brand, phone_type, storage_and_memory, is_5g, color = get_phone_attribute_france(txt)
            # print(brand, phone_type, storage_and_memory, is_5g, color)
            brand_place = "C" + str(i+1)
            phone_type_place = "D" + str(i+1)
            storage_and_memory_place = "E" + str(i+1)
            is_5g_palce = "F" + str(i+1)
            color_place = "G" + str(i+1)
            price_place = "H" + str(i+1)

            sheet[brand_place] = brand
            sheet[phone_type_place] = phone_type
            sheet[storage_and_memory_place] = storage_and_memory
            sheet[is_5g_palce] = is_5g
            sheet[color_place] = color
            sheet[price_place] = price
        else:
            print("france badcase in line {}!!!".format(i))

    book.save(file_name)

def create_new_sheet_france(book,new_sheet,file_name):
    if "france_test" in book.sheetnames:
        book.remove(book["france_test"])
    ws = book.create_sheet('france_test')
    ws["A1"],ws["B1"],ws["C1"],ws["D1"],ws["E1"] = "BRAND", "TYPE", "RAM_AND_STORAGE", "TIME", "PRICE" 
    ws["F1"],ws["G1"],ws["H1"],ws["I1"],ws["J1"],ws["K1"],ws["L1"] = "GERMAN","FRANCE","UK","ITALY","SPAIN","FINLAND","CZECH"

    # new_france = book["法国Darty 01.17"]
    #print(new_france.max_row,new_france.max_column) 

    france_phone = set()
    j = 2
    for i,row in enumerate(new_sheet.values):
        if i > 0:
            brand, phone_type, storage, price, color, is_5g = row[2],row[3],row[4],row[7],row[6],bool(row[5])
            if brand and phone_type and storage and price:
                if is_5g and "5g" not in phone_type:
                    phone_type = phone_type + " 5g "
                
                # new_info = [brand, phone_type, storage, color, price] if color else [brand, phone_type, storage, price]
                # unique_phone_id  = " ".join(new_info)
                unique_phone_id  = " ".join([brand, phone_type, storage, str(price)])
                # print(unique_phone_id)

                # unique_phone_id = " "brand + " " + phone_type + " " + storage + " " + price + " "

                # if unique_phone_id not in france_phone:
                brand_place = "A" + str(j)
                phone_type_place = "B" + str(j)
                storage_and_memory_place = "C" + str(j)
                price_place = "G" + str(j)
                
                ws[brand_place] = brand
                ws[phone_type_place] = phone_type
                ws[storage_and_memory_place] = storage
                ws[price_place] = price
                j += 1
                
                # france_phone.add(unique_phone_id)
    # print(len(france_phone))
    book.save(file_name)